In [5]:
from codex_test import dd2tsv
import re
import numpy as np
import pandas as pd
import logging

logger = logging.getLogger(__name__)

dd2tsv('ijcnlp_dailydialog', 'dd.tsv')
df = pd.read_csv('dd.tsv', sep='\t', header=0)

Things aren't adding up in line 672.
  Turn count: 12
  Act count: 11
  Feel count: 11



In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist

turns = list(turn for turn in df.turn.values.tolist())
sents = []
tokens = []

for turn in turns:
    turn_sents = list(sent for sent in sent_tokenize(turn))
    turn_tokens = list(token for token in word_tokenize(turn))
    for sent in turn_sents:
        sents.append(sent)
    for token in turn_tokens:
        tokens.append(token)

fd_sents = FreqDist()
for sent in sents:
    fd_sents[sent] += 1
        
fd_tokens = FreqDist()
for token in tokens:
    fd_tokens[token] += 1

u_sents = set(sents)
u_tokens = set(tokens)

print(f'Sents: {len(sents)}\nTokens: {len(tokens)}')
print(f'Unique sents: {len(u_sents)}\nVocab: {len(u_tokens)}')

Sents: 166254
Tokens: 1437870
Unique sents: 114967
Vocab: 26362


In [13]:
from random import sample
hapaxes = set(token for token in fd_tokens.hapaxes())

print(len(hapaxes))
print(len(hapaxes)/len(tokens))
print("{:.1%}".format(len(hapaxes)/len(tokens)))
print(sample(hapaxes, 8))

10246
0.007125818050310529
0.7%
['trip-the', 'course.Would', 'Warriors', 'Isis', 'Dial-up', 'totality', 'KC', 'suture']


In [14]:
from nltk.corpus import stopwords
from string import punctuation

stopset = set(stopwords.words('english'))
print(len(stopset))

stopset.update(hapaxes)
print(len(stopset))

punctuation = set(char for char in punctuation)
stopset.update(punctuation)
print(len(stopset))


179
10425
10456


In [30]:
tokenized = list(word_tokenize(sent) for sent in sents)
print(tokenized[100])

filtered_sents = []
for sent in sents:
    tokenized = list(word_tokenize(sent))
    filtered = list(token for token in tokenized if token not in stopset)
    filtered_sents.append(filtered)


['Would', 'you', 'care', 'for', 'a', 'drink', 'before', 'you', 'order', '?']


In [16]:
from gensim.corpora import Dictionary

id2word = Dictionary(filtered_sents)
print(id2word)

Dictionary(15947 unique tokens: ['The', 'kitchen', 'stinks', "'ll", 'I']...)


In [17]:
l = list(i for i in range(15000))
print(id2word[sample(l,1)[0]])

mostly


In [19]:
corpus = list(id2word.doc2bow(tokens) for tokens in filtered_sents)

In [20]:
print(len(corpus))

166254


In [21]:
from gensim import corpora
from gensim.models import Phrases
corpora.MmCorpus.serialize('dd.mm', corpus)

In [22]:
from gensim import models
from gensim import similarities

lsi = models.LsiModel(corpus, id2word=id2word, num_topics=10)
corpus = corpora.MmCorpus('dd.mm')


In [23]:
from gensim import similarities
similarities.Similarity, similarities.MatrixSimilarity
index = similarities.MatrixSimilarity(lsi[corpus])
index.save('dd.index')


In [27]:
similarities.MatrixSimilarity.load('dd.index')

from codex_test import BookBinder
from random import sample

wild = BookBinder('wild.txt')
wild = list(doc.strip('\n') for doc in next(wild))
test = sample(wild, 10)

for doc in test:
    vec = lsi[id2word.doc2bow(doc.lower().split())]
    sims = index[vec]
    sims = sorted(enumerate(sims), key=lambda x: -x[1])[0:10]